# Análisis de detectores

In [2]:
PWD = !pwd
PWD=PWD[0]
WORKSPACE_PATH = PWD+"/../../"
WORKSPACE_DATA_PATH = WORKSPACE_PATH+"/data/"
!ls {WORKSPACE_PATH}/data

datasets  media  models  other	output


In [21]:
!ls {WORKSPACE_PATH}/data/models/

feature_extractor_ll.tar.gz  kaggle-fisheries-yolo  README.md
feature_extractor_mars	     mirtar-yolo


In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import cv2
import os
import time

## YOLOv4 entrenado con kaggle-fisheries

In [3]:
!ls {WORKSPACE_PATH}/data/models/kaggle-fisheries-yolo/

checkpoints			obj.names
kaggle-fisheries-eval.data	README.md
kaggle-fisheries-yolo4.cfg	yolo4-model-performance-report.txt
kaggle-fisheries-yolo4.weights


In [4]:
!cat {WORKSPACE_PATH}/data/models/kaggle-fisheries-yolo/README.md




Tamaño de celda de entrada: 416px

### Preparación de datos

- Cuadernos de preparación de dataset:
    - [Obtención de dataset](../../data-preparation/kaggle-fisheries/01%20-%20Dataset%20download.ipynb)
    - [EDA](../../data-preparation/kaggle-fisheries/02%20-%20Dataset%20exploration.ipynb)
    - [Preparación](../../data-preparation/kaggle-fisheries/03%20-%20Dataset%20preparation.ipynb)

### Arquitectura de la red

In [5]:
!cat {WORKSPACE_PATH}/data/models/kaggle-fisheries-yolo/kaggle-fisheries-yolo4.cfg

[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=32
width=416
height=416
channels=3
momentum=0.949
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 14000
policy=steps
steps=11200.0,12600.0
scales=.1,.1

#cutmix=1
#mosaic=1 Deshabilitado en conda

#:104x104 54:52x52 85:26x26 104:13x13 for 416

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64


### Reporte de entrenamiento

- Curvas no disponibles, pendiente resolver parsing de logs con Darknet.

### Reporte de evaluación

Nota: por escasez de datos, se usó la misma partición para validación que para evaluación.

In [7]:
!cat {WORKSPACE_PATH}/data/models/kaggle-fisheries-yolo/yolo4-model-performance-report.txt

net.optimized_memory = 0 
mini_batch = 1, batch = 32, time_steps = 1, train = 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 268 K-images (4 Kilo-batches_64) 

 calculation mAP (mean average precision)...

 detections_count = 6322, unique_truth_count = 3142  
class_id = 0, name = alb, ap = 90.13%   	 (TP = 1590, FP = 198) 
class_id = 1, name = bet, ap = 66.96%   	 (TP = 150, FP = 35) 
class_id = 2, name = dol, ap = 76.72%   	 (TP = 68, FP = 10) 
class_id = 3, name = lag, ap = 92.08%   	 (TP = 73, FP = 3) 
class_id = 4, name = shark, ap = 94.63%   	 (TP = 129, FP = 11) 
class_id = 5, name = yft, ap = 90.59%   	 (TP = 495, FP = 91) 
class_id = 6, name = other, ap = 90.95%   	 (TP = 233, FP = 46) 

 for conf_thresh = 0.25, precision = 0.87, recall = 0.87, F1-score = 0.87 
 for conf_thresh = 0.25, TP = 2738, FP = 394, FN = 404, average IoU = 69.08 % 

 IoU threshold = 50 %, used 101 Recall

#### Observaciones

mAP@0.50: 86.01 %  (se cumple requerimiento).

In [13]:
df_metrics = pd.DataFrame([
    { "class_id":0, "name": "alb", "ap": 90.13, "TP": 1590, "FP": 198 },
    { "class_id":1, "name": "bet", "ap": 66.96, "TP": 150, "FP": 35 },
    { "class_id":2, "name": "dol", "ap": 76.72, "TP": 68, "FP": 10 },
    { "class_id":3, "name": "lag", "ap": 92.08, "TP": 73, "FP": 3 },
    { "class_id":4, "name": "shark", "ap": 94.63, "TP": 129, "FP": 11 }, 
    { "class_id":5, "name": "yft", "ap": 90.59, "TP": 495, "FP": 91 },
    { "class_id":6, "name": "other", "ap": 90.95, "TP": 233, "FP": 46 }
])
df_metrics.sort_values("ap",ascending=False)

,class_id,name,ap,TP,FP
4,4,shark,94.63,129,11
3,3,lag,92.08,73,3
6,6,other,90.95,233,46
5,5,yft,90.59,495,91
0,0,alb,90.13,1590,198
2,2,dol,76.72,68,10
1,1,bet,66.96,150,35


## YOLOv4 entrenado con Mirtar (aumentado)

In [22]:
!ls {WORKSPACE_PATH}/data/models/mirtar-yolo/

checkpoints	  mirtar-yolo4.weights	README.md
mirtar-yolo4.cfg  obj.names		yolo4-model-performance-report.txt


In [23]:
!cat {WORKSPACE_PATH}/data/models/mirtar-yolo/README.md




Tamaño de celda de entrada: 416px

### Preparación de datos

- Cuadernos de preparación de dataset:
    - [Preparacíón de dataset c/ aumentado](../../data-preparation/01%20-%20Dataset%20preparation.ipynb)

### Arquitectura de la red

In [30]:
!cat {WORKSPACE_PATH}/data/models/mirtar-yolo/mirtar-yolo4.cfg

[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.949
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 6000
policy=steps
steps=4800.0,5400.0
scales=.1,.1

#cutmix=1
#mosaic=1 Deshabilitado en conda

#:104x104 54:52x52 85:26x26 104:13x13 for 416

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
siz

### Reporte de entrenamiento

- Curvas no disponibles, pendiente resolver parsing de logs con Darknet.

### Reporte de evaluación

Nota: por escasez de datos, se usó la misma partición para validación que para evaluación. 

In [16]:
!cat {WORKSPACE_PATH}/data/models/mirtar-yolo/yolo4-model-performance-report.txt

net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 294 K-images (4 Kilo-batches_64) 

 calculation mAP (mean average precision)...

 detections_count = 203, unique_truth_count = 116  
class_id = 0, name = operador, ap = 93.48%   	 (TP = 73, FP = 10) 
class_id = 1, name = red, ap = 78.85%   	 (TP = 5, FP = 0) 
class_id = 2, name = pescado, ap = 63.03%   	 (TP = 22, FP = 7) 

 for conf_thresh = 0.25, precision = 0.85, recall = 0.86, F1-score = 0.86 
 for conf_thresh = 0.25, TP = 100, FP = 17, FN = 16, average IoU = 63.76 % 

 IoU threshold = 50 %, used 101 Recall-points 
 mean average precision (mAP@0.50) = 0.784539, or 78.45 % 

Set -points flag:
 `-points 101` for MS COCO 
 `-points 11` for PascalVOC 2007 (uncomment `difficult` in voc.data) 
 `-points 0` (AUC) for ImageNet, PascalVOC 2010-2012, your custom datase

#### Observaciones

mAP@0.50 = 78.45 % (se cumple requerimiento).

In [28]:
df_metrics = pd.DataFrame([
    { "class_id": 0, "name": "operador", "ap": 93.48, "TP": 73, "FP": 10 },
    { "class_id": 1, "name": "red", "ap": 78.85,  "TP": 5, "FP": 0 },
    { "class_id": 2, "name": "pescado", "ap": 63.03,  "TP": 22, "FP": 7 }
])
df_metrics.sort_values("ap",ascending=False)

,class_id,name,ap,TP,FP
0,0,operador,93.48,73,10
1,1,red,78.85,5,0
2,2,pescado,63.03,22,7
